In [21]:
import os
import subprocess
import numpy as np
import tqdm
import time
import datetime

#if you want to display images as you record
import cv2
import matplotlib.pyplot as plt

from multiprocessing import Queue, Event

from pyk4a import *
from top_bottom_triggered.video_io import *
from top_bottom_triggered.multicam_utils import *

In [22]:
import glob

port = glob.glob("/dev/ttyACM*0")[0]
print(port)

IndexError: list index out of range

# Setup

In [23]:
# set recording params
name = '3_09'
file_prefix = f'/home/dattalab1/Desktop/DOI/{name}/{name}'

In [24]:
serial_numbers = {
    'top': '000438621712'
}

recording_length = int(60*20)

camera_indexes = get_camera_indexes(serial_numbers)
print(camera_indexes)

Index:0	Serial:000438621712	Color:1.6.110	Depth:1.6.80
{'top': 0}


In [25]:
record_processes = {}
sync_delay,sync_delay_step = 0,500
interrupt_queues = {camera: Queue() for camera,ix in camera_indexes.items()}
trigger_started_event = Event()

for camera,ix in camera_indexes.items():
    sync_delay += sync_delay_step
    k4a = PyK4A(Config(color_resolution=ColorResolution.OFF,
                       depth_mode=DepthMode.NFOV_UNBINNED,
                       synchronized_images_only=False,
                       wired_sync_mode=WiredSyncMode.SUBORDINATE,
                       subordinate_delay_off_master_usec=sync_delay), device_id=ix)

    p = Process(target=capture_from_azure, 
                args=(k4a, file_prefix+'.'+camera ,recording_length+5),
                kwargs={
                    'display_time': True,
                    'display_frames': True,
                    'externally_triggered': True,
                    'trigger_started_event': trigger_started_event,
                    'interrupt_queue': interrupt_queues[camera]
                })

    record_processes[camera] = p

    

## Run this cell before you record!

In [26]:
# Prime the Azure   
record_processes[camera].start()
time.sleep(3)
print('Go ahead!')

awaiting first capture (trigger)...
Go ahead!
First frame captured!


Qt: Session management error: None of the authentication protocols specified are supported


Recorded 1187 out of 1205 seconds
Stopping process for 3_09.top
 - Frame rate =  29.50964544599243


In [9]:
# DEBUG: run this to see if azures received triggers

trigger_started_event.is_set()

True

## Now run the basler script to start data acquisition...

In [27]:
# empty space

In [28]:
# empty space

In [6]:
trigger_started_event.is_set()

False

libusb: error [udev_hotplug_event] ignoring udev action bind
libusb: error [udev_hotplug_event] ignoring udev action bind


## Run this cell when you're done

In [1]:
if trigger_started_event.is_set():
    exit_codes = [p.join() for p in record_processes.values() if p.is_alive()]
    print('Finished saving recordings...')

NameError: name 'trigger_started_event' is not defined

In [2]:
device_timestamps = [np.load(f'{file_prefix}.{cam}.device_timestamps.npy') for cam in serial_numbers]
print(len(device_timestamps[0]))

NameError: name 'serial_numbers' is not defined

## Debugging: run this cell to interrupt azures

In [19]:
from glob import glob
glob('/dev/ttyAC*')

['/dev/ttyACM2', '/dev/ttyACM1']

In [20]:
stop_azures(trigger_started_event, interrupt_queues, '/dev/ttyACM2')


halting azures
Stopping sync device


In [ ]:
plt.hist(np.diff(device_timestamps[0]),bins=10)
plt.hist(np.diff(device_timestamps[1]),bins=10)

In [ ]:
sum(np.diff(device_timestamps[1])>40000)

In [ ]:
sum(np.diff(device_timestamps[0])>40000)

In [ ]:
#import matploli
depth = read_frames(file_prefix+'.top.ir.avi', [101], pixel_format='gray16').squeeze()
plt.imshow(depth, vmax=1000)

